In [1]:
import sys
import logging
import numpy as np
from joblib import hash, dump, load
import os

from deer.default_parser import process_args
from deer.agent import NeuralAgent
from deer.learning_algos.CRAR_torch import CRAR
from simple_maze_env import MyEnv as simple_maze_env
import deer.experiment.base_controllers as bc

from deer.policies import EpsilonGreedyPolicy

In [2]:
class Defaults:
    # ----------------------
    # Experiment Parameters
    # ----------------------
    steps_per_epoch = 5000
    epochs = 50
    steps_per_test = 1000
    period_btw_summary_perfs = 1
    
    # ----------------------
    # Environment Parameters
    # ----------------------
    frame_skip = 2

    # ----------------------
    # DQN Agent parameters:
    # ----------------------
    update_rule = 'rmsprop'
    learning_rate = 5 * 1E-4 # 1E-4
    learning_rate_decay = 0.9
    discount = 0.9
    discount_inc = 1
    discount_max = 0.99
    rms_decay = 0.9
    rms_epsilon = 0.0001
    momentum = 0
    clip_norm = 1.0
    epsilon_start = 1.0
    epsilon_min = 1.0
    epsilon_decay = 10000
    update_frequency = 1
    replay_memory_size = 1000000 #replacing with 200000 will works just fine (in case you dont have 18gb of memory)
    batch_size = 32
    freeze_interval = 1000
    deterministic = False

higher_dim_obs = True

In [3]:

parameters = Defaults()
if parameters.deterministic:
    rng = np.random.RandomState(123456)
else:
    rng = np.random.RandomState()

# --- Instantiate environment ---
env = simple_maze_env(rng, higher_dim_obs=higher_dim_obs)

# --- Instantiate learning_algo ---
learning_algo = CRAR(
    env,
    parameters.rms_decay,
    parameters.rms_epsilon,
    parameters.momentum,
    parameters.clip_norm,
    parameters.freeze_interval,
    parameters.batch_size,
    parameters.update_rule,
    rng,
    high_int_dim=False,
    internal_dim=2, lr=parameters.learning_rate)

test_policy = EpsilonGreedyPolicy(learning_algo, env.nActions(), rng, 1.)

# --- Instantiate agent ---
agent = NeuralAgent(
    env,
    learning_algo,
    parameters.replay_memory_size,
    max(env.inputDimensions()[i][0] for i in range(len(env.inputDimensions()))),
    parameters.batch_size,
    rng,
    test_policy=test_policy)

# --- Create unique filename for FindBestController ---
h = hash(vars(parameters), hash_name="sha1")
fname = "test_" + h
print("The parameters hash is: {}".format(h))
print("The parameters are: {}".format(parameters))

# As for the discount factor and the learning rate, one can update periodically the parameter of the epsilon-greedy
# policy implemented by the agent. This controllers has a bit more capabilities, as it allows one to choose more
# precisely when to update epsilon: after every X action, episode or epoch. This parameter can also be reset every
# episode or epoch (or never, hence the resetEvery='none').
agent.attach(bc.EpsilonController(
    initial_e=parameters.epsilon_start,
    e_decays=parameters.epsilon_decay,
    e_min=parameters.epsilon_min,
    evaluate_on='action',
    periodicity=1,
    reset_every='none'))

agent.run(10, 500)
print("end gathering data")

# --- Bind controllers to the agent ---
# Before every training epoch (periodicity=1), we want to print a summary of the agent's epsilon, discount and 
# learning rate as well as the training epoch number.
agent.attach(bc.VerboseController(
    evaluate_on='epoch', 
    periodicity=1))

# Every epoch end, one has the possibility to modify the learning rate using a LearningRateController. Here we 
# wish to update the learning rate after every training epoch (periodicity=1), according to the parameters given.
agent.attach(bc.LearningRateController(
    initial_learning_rate=parameters.learning_rate, 
    learning_rate_decay=parameters.learning_rate_decay,
    periodicity=1))

# Same for the discount factor.
agent.attach(bc.DiscountFactorController(
    initial_discount_factor=parameters.discount, 
    discount_factor_growth=parameters.discount_inc, 
    discount_factor_max=parameters.discount_max,
    periodicity=1))

# During training epochs, we want to train the agent after every [parameters.update_frequency] action it takes.
# Plus, we also want to display after each training episode (!= than after every training) the average bellman
# residual and the average of the V values obtained during the last episode, hence the two last arguments.
agent.attach(bc.TrainerController(
    evaluate_on='action', 
    periodicity=parameters.update_frequency, 
    show_episode_avg_V_value=True, 
    show_avg_Bellman_residual=True))

# We wish to discover, among all versions of our neural network (i.e., after every training epoch), which one 
# has the highest validation score.
# To achieve this goal, one can use the FindBestController along with an InterleavedTestEpochControllers. It is 
# important that the validationID is the same than the id argument of the InterleavedTestEpochController.
# The FindBestController will dump on disk the validation scores for each and every network, as well as the 
# structure of the neural network having the best validation score. These dumps can then used to plot the evolution 
# of the validation and test scores (see below) or simply recover the resulting neural network for your 
# application.
agent.attach(bc.FindBestController(
    validationID=simple_maze_env.VALIDATION_MODE,
    testID=None,
    unique_fname=fname))

# All previous controllers control the agent during the epochs it goes through. However, we want to interleave a 
# "validation epoch" between each training epoch. For each validation epoch, we want also to display the sum of all 
# rewards obtained, hence the showScore=True. Finally, we want to call the summarizePerformance method of ALE_env 
# every [parameters.period_btw_summary_perfs] *validation* epochs.
agent.attach(bc.InterleavedTestEpochController(
    id=simple_maze_env.VALIDATION_MODE, 
    epoch_length=parameters.steps_per_test,
    periodicity=1,
    show_score=True,
    summarize_every=1))

The parameters hash is: 62977be8e45d8a56a5537c11dfd5d2fd8dda69e0
The parameters are: <__main__.Defaults object at 0x2ae487dc08b0>
end gathering data


In [4]:
try:
    os.mkdir("params")
except Exception:
    pass
dump(vars(parameters), "params/" + fname + ".jldump")
agent.gathering_data=False
agent.run(parameters.epochs, parameters.steps_per_epoch)

# --- Show results ---
basename = "scores/" + fname
scores = load(basename + "_scores.jldump")
print (scores)

Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([-0.3075, -0.0738], device='cuda:0') tensor([-0.3935, -0.3524], device='cuda:0') tensor([-0.3075, -0.0738], device='cuda:0')
R[0]
tensor([0.2149], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.lossR/500., self.loss_gamma/500., self.loss_Q/500., self.loss_disentangle_t/500., self.loss_disambiguate1/500., self.loss_disambiguate2/500.
0.03939761593192816 0.06175614380836487 1.1208807349205017 0.0 0.3985723863840103 0.0007902259081602097 0.0944376584906131
self.loss_interpret/500.
0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
2 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([ 0.5836, -0.9203], device='cuda:0') tensor([ 0.6992, -0.5662], device='cuda:0') tensor([ 0.8374, -0.4729], device='cuda:0')
R[0]
tensor([0.2582], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.lossR

/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:226: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  line3 = ax.scatter(all_possib_abs_states[length_block[i][0]:length_block[i][1],0], all_possib_abs_states[length_block[i][0]:length_block[i][1],1], c=colors[i], marker='x', edgecolors='k', alpha=0.5, s=100)
/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:269: RuntimeWarning: divide by zero encountered in log
  plt.plot(np.log(learning_algo.tracked_disamb1))


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([-0.6749, -0.1739], device='cuda:0') tensor([-0.6453,  0.2037], device='cuda:0') tensor([-0.6165,  0.2999], device='cuda:0')
R[0]
tensor([0.2096], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.lossR/500., self.loss_gamma/500., self.loss_Q/500., self.loss_disentangle_t/500., self.loss_disambiguate1/500., self.loss_disambiguate2/500.
0.00721187219908461 0.6784051927104592 1.126071973323822 0.0 0.36924794697761537 0.0002430214136838913 0.05153244132269174
self.loss_interpret/500.
0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([0.5488, 0.3806], device='cuda:0') tensor([0.5429, 0.6510], device='cuda:0') tensor([0.5047, 0.7238], device='cuda:0')
R[0]
tensor([0.2277], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.lossR/500.,

/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:226: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  line3 = ax.scatter(all_possib_abs_states[length_block[i][0]:length_block[i][1],0], all_possib_abs_states[length_block[i][0]:length_block[i][1],1], c=colors[i], marker='x', edgecolors='k', alpha=0.5, s=100)
/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:269: RuntimeWarning: divide by zero encountered in log
  plt.plot(np.log(learning_algo.tracked_disamb1))


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([-0.0446,  0.6119], device='cuda:0') tensor([-0.0370,  0.6560], device='cuda:0') tensor([-0.0446,  0.6119], device='cuda:0')
R[0]
tensor([0.2577], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.lossR/500., self.loss_gamma/500., self.loss_Q/500., self.loss_disentangle_t/500., self.loss_disambiguate1/500., self.loss_disambiguate2/500.
0.0030015448874328287 1.294117451198399 1.1255337076187133 0.0 0.35560623943805697 0.00019803528487682344 0.054517265439033506
self.loss_interpret/500.
0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([-0.9752,  0.9720], device='cuda:0') tensor([-0.9678,  0.9525], device='cuda:0') tensor([-0.9752,  0.9720], device='cuda:0')
R[0]
tensor([0.2405], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.l

...

/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:226: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  line3 = ax.scatter(all_possib_abs_states[length_block[i][0]:length_block[i][1],0], all_possib_abs_states[length_block[i][0]:length_block[i][1],1], c=colors[i], marker='x', edgecolors='k', alpha=0.5, s=100)
/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:269: RuntimeWarning: divide by zero encountered in log
  plt.plot(np.log(learning_algo.tracked_disamb1))


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([9.9885e-01, 9.0645e-05], device='cuda:0') tensor([0.9919, 0.3822], device='cuda:0') tensor([0.9986, 0.3860], device='cuda:0')
R[0]
tensor([0.2370], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.lossR/500., self.loss_gamma/500., self.loss_Q/500., self.loss_disentangle_t/500., self.loss_disambiguate1/500., self.loss_disambiguate2/500.
0.00022771277610445396 10.566384330578149 1.1269272525310516 0.0 0.32958226472139357 1.7666429281234742e-05 0.054265123667195436
self.loss_interpret/500.
0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([0.3715, 0.5873], device='cuda:0') tensor([0.2396, 0.0293], device='cuda:0') tensor([0.2236, 0.0430], device='cuda:0')
R[0]
tensor([0.2685], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.los

/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:226: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  line3 = ax.scatter(all_possib_abs_states[length_block[i][0]:length_block[i][1],0], all_possib_abs_states[length_block[i][0]:length_block[i][1],1], c=colors[i], marker='x', edgecolors='k', alpha=0.5, s=100)
/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/simple_maze_env.py:269: RuntimeWarning: divide by zero encountered in log
  plt.plot(np.log(learning_algo.tracked_disamb1))


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([ 0.9981, -0.9993], device='cuda:0') tensor([ 0.9965, -0.4735], device='cuda:0') tensor([ 0.9977, -0.4615], device='cuda:0')
R[0]
tensor([0.2408], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self.lossR/500., self.loss_gamma/500., self.loss_Q/500., self.loss_disentangle_t/500., self.loss_disambiguate1/500., self.loss_disambiguate2/500.
0.00020472394020180218 11.184885167099536 1.1258387310504914 0.0 0.32596287813782693 1.9441775977611542e-05 0.052152340402826666
self.loss_interpret/500.
0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
2 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([-0.3804,  0.2128], device='cuda:0') tensor([-0.3848,  0.2136], device='cuda:0') tensor([-0.3804,  0.2128], device='cuda:0')
R[0]
tensor([0.2379], device='cuda:0', grad_fn=<SelectBackward0>)
self.loss_T/500., self


KeyboardInterrupt

